In [1]:
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import copy
import torch as tr

# import scripts
from src.es import random_opt
from src.es import evolutionary_strategy
from src.grad_opt import ADAM_opt
from src.api_helper import api_utils
from src.api_helper import env
from src.helper import helper

device = tr.device("cuda" if tr.cuda.is_available() else "cpu")
dtype = tr.float32

##### data

In [2]:
## multi-process has problem with decorator
# def af(query): return env.rosenbrock(query); api = api_utils.multi_process_transform(af) 

@api_utils.transform
def api(query): return env.rosenbrock(query)

# @api_utils.multi_thread_transform
# def api(query): return env.rosenbrock(query)

x0, y0 = tr.tensor([4,4], dtype=dtype).view(1, -1), tr.tensor([[-1]], dtype=dtype).view(-1, 1)  
r0 = env.rosenbrock(x0)  # initial reward on which normalisation based

x0, y0= x0.to(device), y0.to(device)  # data in GPU

print("init query:", x0); print(f"init reward: {api(x0, r0, device)}, origin reward: {r0.item():.2f}")  # x,y stay on GPU while r0 in cpu



init query: tensor([[4., 4.]])
init reward: tensor([[-1.]]), origin reward: 14409.00


##### OPT

In [3]:
# hyperparameters
T = 1000  # total number of iterations

adam = ADAM_opt(); ram_opt = random_opt(); es = evolutionary_strategy()

In [4]:
%%time
x,y = es.outer_loop(T, x0, r0, api)

CPU times: user 11.7 s, sys: 11.9 ms, total: 11.7 s
Wall time: 11.8 s


In [5]:
# saver.standard_save(x,y,r0, "es", "./data/es.pt")

saving es data


In [10]:
%%time
x2, y2 = ram_opt.outer_loop(T, (-4,4), x0, r0, api, batch_size=2)

CPU times: user 315 ms, sys: 2.98 ms, total: 318 ms
Wall time: 320 ms


In [11]:
# saver.standard_save(x2, y2, r0, "random", "./data/random.pt")

saving random data


In [8]:
%%time
x1,y1 = adam.outer_loop(T, x0, r0, api, env.rosenbrock_grad)

CPU times: user 389 ms, sys: 18 µs, total: 389 ms
Wall time: 390 ms


In [9]:
# saver.standard_save(x1,y1,r0, "adam", "./data/adam.pt")

saving adam data
